## 1. Import Liberaries

In [1]:
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils

import warnings
warnings.filterwarnings('ignore')

## 1.1 Load the model

In [2]:
IMAGE_SIZE = (12, 8) # Output display size as you want
PATH_TO_SAVED_MODEL="inference_graph\\saved_model\\"
print('Loading model...', end='')


# start= time.time()
# Load saved model and build the detection function
detect_fn=tf.saved_model.load(PATH_TO_SAVED_MODEL)

print('Done!')


Loading model...Done!


## 2. Load and convert the image into tensor and pass to model 

In [67]:
from PIL import Image
import numpy as np


category_index=label_map_util.create_category_index_from_labelmap("labelmap.pbtxt",use_display_name=True)

def load_image_into_numpy_array(path):

    return np.array(Image.open(path))

image_path = "2Jbdy.jpg"

image_np = load_image_into_numpy_array(image_path)

input_tensor = tf.convert_to_tensor(image_np)

input_tensor = input_tensor[tf.newaxis, ...]

detections = detect_fn(input_tensor)

num_detections = int(detections.pop('num_detections'))
detections = {key: value[0, :num_detections].numpy()
              for key, value in detections.items()}

# threshold=0.3
# classes = [cls for cls in detections['detection_classes'][detections['detection_scores'] > threshold]]
# classes = [category_index.get(cls)['name'] for cls in classes]

# classes

In [68]:
# detections.keys()

In [69]:
boxes=detections['detection_boxes']
classes=detections['detection_classes']
scores=detections['detection_scores']

## Loop through model predictions and set minimum confidence score

In [70]:
captcha_array = []
for i,b in enumerate(boxes):
    for Symbol in range(63):
        if classes[i] == Symbol: # check if detected class equal to our symbols
            if scores[i] >= 0.30: # do something only if detected score more han 0.30
                                            # x-left        # x-right
                mid_x = (boxes[i][1]+boxes[i][3])/2 # find x coordinates center of letter
            # to captcha_array array save detected Symbol, middle X coordinates and detection percentage
                captcha_array.append([category_index[Symbol].get('name'), mid_x, scores[i]])

### Again validatating incase output is less than length of 5

In [71]:
if len(captcha_array) <5:
    captcha_array = []
    for i,b in enumerate(boxes):
        for Symbol in range(63):
            if classes[i] == Symbol: # check if detected class equal to our symbols
                if scores[i] >= 0.20: # do something only if detected score more han 0.20
                                            # x-left        # x-right
                    mid_x = (boxes[i][1]+boxes[i][3])/2 # find x coordinates center of letter
            # to captcha_array array save detected Symbol, middle X coordinates and detection percentage
                    captcha_array.append([category_index[Symbol].get('name'), mid_x, scores[i]])
    

## Preserving order/sequence of the output

In [72]:
for number in range(20):
    for captcha_number in range(len(captcha_array)-1):
        if captcha_array[captcha_number][1] > captcha_array[captcha_number+1][1]:
            temporary_captcha = captcha_array[captcha_number]
            captcha_array[captcha_number] = captcha_array[captcha_number+1]
            captcha_array[captcha_number+1] = temporary_captcha

### Calulating average error distrance and validating

In [73]:
average_distance_error=3
average = 0
captcha_len = len(captcha_array)-1
while captcha_len > 0:
    average += captcha_array[captcha_len][1]- captcha_array[captcha_len-1][1]
    captcha_len -= 1


average = average/(len(captcha_array)+average_distance_error)

captcha_array_filtered = list(captcha_array)
captcha_len = len(captcha_array)-1
while captcha_len > 0:
    if captcha_array[captcha_len][1]- captcha_array[captcha_len-1][1] < average:
        if captcha_array[captcha_len][2] > captcha_array[captcha_len-1][2]:
            del captcha_array_filtered[captcha_len-1]
        else:
            del captcha_array_filtered[captcha_len]
    captcha_len -= 1

# Get final string from filtered CAPTCHA array
captcha_string = ""
for captcha_letter in range(len(captcha_array_filtered)):
    captcha_string += captcha_array_filtered[captcha_letter][0]

In [74]:
captcha_string

'2Jbdy'